In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

from selenium.webdriver.common.keys import Keys

In [2]:
name=['플로랑향기뿜뿜']
category=['별점']

# 플로랑향기뿜뿜 디퓨저 후기
ns_address="https://search.shopping.naver.com/catalog/30028674618?query=%EB%94%94%ED%93%A8%EC%A0%80&NaPm=ct%3Dlbavbxcg%7Cci%3D80f9ef2b15499c0ef0dce68d77c4051793e838c3%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Dd6f58c0674cf8848d341e83a809e9982455ec90c"
#xpath
shoppingmall_review="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[1]/ul/li[3]/a"
category1="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[2]/a" #5점
category2="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[3]/a" #4점
category3="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[4]/a" #3점
category4="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[5]/a" #2점
category5="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/div[2]/ul/li[6]/a" #1점

In [3]:
#함수 선언
def add_dataframe(name,category,reviews,stars,dates,options,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star','date','option'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i],dates[i],options[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,date,option,'null','null']
        n+=1    
    return df1

In [4]:
# 웹페이지 열기

header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_13888/2638303677.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
# 5점 리뷰 가져오기
d.find_element(By.XPATH,category1).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
dates=[]
options=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    #sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            date=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[4]').text
            dates.append(date)
            option=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[5]').text
            options.append(option)
            #if j%2==0: #화면에 2개씩 보이도록 스크롤
            #    ELEMENT = d.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
            #    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review, star, date, option, "\n")
            cnt+=1 
        except: break
            
        #sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
        
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[12]').click()
            else : next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
#save()

페이지 1 

1 제가 캔들,디퓨저 만드는 사람입니다~
디퓨져는 사서는 써본적없고 받아서만 써보았어요,이번에 한번 사서 쓰게 되어보았어요. 제가 만들어 쓰는향들은 하두 익숙해져서 향이 엄청 강한향을 좋아라하니..색다른 향은 냄새를 금방 느끼는데 늘 맡던 향엔 코가 적응 하였는지 많이 안나는거같더라구요. 저에겐 없는 청포도 향을 사보았어요. 글고 빨래냄새~ 빨래냄새 일반 제가 쓰는 용액 다우니보단 향은 덜해요. (저는 향을 하도 많이 맡아서그런데 보통사람들에겐 진할수도있어요) 청포도향은 처음 맡는 향이라 진하게 느껴지고 실제로도 다른향보단 강한거같제요. 회사에서 이층에 청포도 해놓으니 1층까지도 발향될정도로 좋아요~ 제 제품과 조금 다른게 있다면 용액 닳는 속도 가 조금 빠른거같기두해요~그래도 발향이 잘되서 좋네여 평점5 21.06.24. 옵션 선택=디퓨저 리필액 1L, 향기 선택=#02_리필액 아리아 1L, 1+1 선택=선택 안함 

2 디퓨저 좋아하는 사람이라 이것저것 많이 옴겨타다가 인스광고보고 혹시나하고 사봤어요.
발향이 좋은지 안좋은지 사실 알송달송해요;;
6개세트 청포도 구매했구요 거실2개
신발장현관1개 중문지나서1개 화장실한개씩 한번에 다 설치했어요.
거실은 향없구요
나름 밀폐된곳 화장실 현관은
스틱 4개씩 꼽아뒀구요.
중문입구1개 식탁위1개 거실장1개 이렇게 나눴는데 디퓨저에 코대봐도 그다지 안나요
그렇다고 향이 없나하기엔 화장실 현관에는 향이 나구요... 강하게나고 그러진 않아요 거실에 향이 퍼지길 바랫는데 씁쓸 34평 일반 아파트입니다...
가격저렴하니 그냥 그러려니하는데 재구매는 안할꺼같아요4
디퓨저 육목민이라... 평점3 22.07.19. 옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

3 가격도 저렴하고
작은 통에 소분해서 여러군데 놔뒀는데
일단 좁은 곳에서는 발향이 확실히
다른 디퓨저보다 잘돼요
현관에 하나 놔뒀는데 현관문 열자마다

거실,주방,안방,옷방 이렇게 놔뒀

25 알바 아니고요 내돈내산 리얼 찐 후기입니다.
솔직히 sns광고 보고 호구해서 몇번 구매했다가 당했다는 느낌이나 실망하거나 그랬는데 이건 진짜 찐이에요. 발향력 진짜 갑오브갑 진짜 짱이에요 거실에 하나 방에 하나놔뒀는데 상품 오자마자 거실엔 스틱 5개 꽂고 방엔 3개 꽂았는데 그 다음날 일끝나고 집에왔는데 냄새에 질식하는줄 알았어요 향기가 진짜 꽉 차있더라구요 그래서 거실엔 스틱 2개 뺐다가 오늘 하나 더 빼봤구요(3개만 꽂아놔도 발향 쩔어요 지금은 두개) 방에는 2개만 꽂아놨습니다 한개만 꽂을까했는데 그럼 안예뻐 보이잖아요 ㅋㅋㅋ 글만쓰면 안믿으실까봐 사진도 찍어올려요 진짜 향도 제가 좋아하는 향으로 잘 골라서 샀네요 ㅠㅠ 용량도 짐승 용량인데 가격도 진짜 너무 착해요 앞으로 이거 계속 쓸꺼에요 계속 이렇게 원플원으로 이 가격으로 계속 팔아주세요오ㅠㅠㅠ 평점5 22.02.17. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#06_디퓨저 몰리스 500ml 

26 ** 아리아 후기 ** 결론부터 말씀드리면 아리아 향 너무 좋아요👍🏻👍🏻 예전에 몰리스랑 아리아로 3개 구매해서 아리아는 남친 집에, 몰리스 1개는 저희 집에서 썼는데, 몰리스는 향 자체가 진하고 쎄서 드레스룸에 스틱을 2개만 넣었놨는데도 온 가족이 다 머리아파 하더라구요ㅠㅠ

아리아는 제 방에 스틱 2개 꽂아놓고 쓰는데~ 섬유유연제로 헹궈 잘 말린 뽀송뽀송한 이불 덮고 자는 느낌이에요!! 엄마도 이번향은 너무 좋다고 하네요 :) 사람마다 취향이 다르겠지만 가족들이 달달한 과일향은 안좋아해서 아리아로만 3개 샀는데 대만족입니다.😍 평점5 22.01.28. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

27 아이들이 너무좋아해요

저는 다른공병에 나눠담다가 쏟아서
냄새를 너무 갑자기 집중적으로 맡았더니
코가 마비되는 느낌이지만
일단 포도향 넘나리달달한게
좋습니당 ~^

페이지 4 

50 빠른배송 너무 감사합니다.
남자들만 득실거리는 건설현장 사무실입니다.
컨테이너로 된 임시 가설 사무실이라 그런지 퀴퀴한냄새+수컷들의 진한향(땀냄새,홀애비냄새,담배냄새 등등)이 어우러져 늘 환기를 열심히해도 잡히지않던 냄새가 오늘 아침에 뚜껑을 열자마자 확 달라졌어요
리드스틱을 3개 꽂았더니 향이 강하게 분출되네요 일반 가정이라면 두개만 꽂아도 충분할것같습니다.
칼퇴할겁니다. 평점5 22.09.23. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#06_디퓨저 아이솝 500ml, 1+1 선택=#06_디퓨저 아이솝 500ml 

51 오자마자 오픈해서 우드스틱 두개씩 넣었는데 진짜 이름처럼 바로 향기뿜뿜 ㅠㅠ진짜 인생 디퓨져 찾았네요.... 디퓨져 금방 써서 저렴이로 (코코xx) 사용했었는데 그건 가까이 가사 코를
대야 향기났는데 이건 온 집안에 향기가 가득하네요, 화장실 거실 방 드레스룸 하나씩 넣었뒀어요!
1+1+1 행사로 저렴하게 득템했네요
리뷰 보고 구매했는데 이거 보시는분들 절대 고민하지말도 사세요 평점5 20.01.11. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=#04_비쥬 500ml+섬유스틱(5개) 

52 누구나 다 좋아하는 향이죠^^ 2개밖에 안 꽂았는데 온 방들이 다 향기로워요 발향력이 진짜 좋은 것 같아요.. 다음에 주문할땐 1개만 꽂는 것도 괜찮을 것 같네요 대용량이라 얼마나 쓸 수 있을지 궁금하기도 하구요 >.< 벨르는 약간 포도 오래된 냄새도 함께나는 것 같아요 그게 매력인걸까요? 상큼시큼한 냄새가 나네요 평점5 22.03.17. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

53 밖에 나갔다 들어오면 우리집 특유의 집 냄세가 싫어서 쿤*, 코코도* 디퓨져 알아보다가 리뷰 좋고 나쁜 성분이 적다고 하길래 구매했어요!
33평에 공기와 바람이 잘 통하는 곳에 스틱 3

84 이주정도 사용 후기입니다!
향이 진짜 좋아요 거실에 하나 안방에 하나 화장실에 하나 각각 다른 향으로 뒀는데 공간마다 새로운 느낌도 좋고 향이 은은한건 은은하고 찐한건 찐하고 다른 디퓨저들은 이주차쯤부터는 향이 안나는데 이건 계속 향이 지속되네요 단점은 정말 빨리 닳아요 그래요 너무너무 맘에 들어서 이건 앞으로 쭈욱 애용할거 같아요 감사합니다 평점5 22.10.29. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

85 풍자언니 리뷰에 혹해서 구매함ㅋㅋ
다른 사람들 리뷰마냥 하나로 30평에 다퍼진다 그정도는 아니지만 다른 디퓨저에 비해 발향력 좋음
포장도 안전하게 잘 되서 옴 매우 만족
여섯개 구매해서 방마다 하나씩 거실양옆에 두개 두니 현관문열자마자 향기가 남 굳굳 평점5 22.04.11. 옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

86 대용량에다가 향도 톡쏘는 거 없이 다 맘에 들어요!!! 근데 택배 상자가 구겨져서왔고 뽁뽁이 안쪽으로 물은 아닌데 꾸리꾸리한 이상한 냄새나는 액체가 묻어있어서 사진과 같이 종이가 좀 젖어서 왔어요 그래서 좀 찝찝한데 그것 빼면 다 좋아요 평점5 22.07.05. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

87 다른 제품들은 쓰면 향도 별로 안나고 금방 줄어들고 만족하는 제품이 하나도 없었는데 플로랑 향기뿜뿜은 진짜~~~너~~~무 좋아요 향도 잘 퍼지고 잘 줄지도 않고 용량도 많고 저번에 써보고도 좋아서 이번에 다른향으로 재주문 했어요 ㅎㅎㅎ 이제 디퓨저는 여기서만 사려구요 평점5 22.06.15. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#03_디퓨저 벨르 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

88 이거 .. 풍자님이 광고 받기 한참전에 방송에서 한번 이야기 

121 배송 빠르고 용량 크고 좋아요~
일단 청포도향부터 뜯어서 써봤는데
생각보다 엄청 진하게 멀리 퍼지는것 같진 않고
은은하게 살짝 나는 정도인거같아요
이게 잘못 맡으면 좀 찌름한 냄새같기도 해서
요정도만 딱 나는게 좋은거같아요 :)
잘 쓰고있습니당!! 평점5 22.02.17. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

122 발향이 강하고 좋네요.
약간 벨르는 청포도 향기 보다 다른 향 인 것 같아요.발향이 너무 강해서 리드스틱 2개만 꽂았는데도 발향이 엄청 강해요.살짝 머리 아프긴 한데 아리아?는 향이 약한 것 같아서 세개 넣고 화장실에 넣어놨더니 향이 이것도 강하네요. 이 가격에 두개 제품을 살 수 있고 발향도 강하고 나쁘지 않네요. 평점5 20.04.29. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#03_벨르 500ml+섬유스틱(5개), 1+1 선택=#02_아리아 500ml+섬유스틱(5개) 

123 방에 식물향 나게 하고 싶어서 풀향 샀다가... 술 먹고 향 맡으니 너무 힘들어서 화장실로 보내고....
비누향 하나 더 구매해서 방에 뒀더니 은은하니 포근하고 좋아요🥹
동생은 다우니향 사달래서 또 샀어요🙋‍♀️
대용량에 향 좋아요🤭 평점5 22.10.23. 옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#06_디퓨저 아이솝 500ml, 1+1 선택=선택 안함 

124 남자친구가 이사를 해서 디퓨저가 필요해서 알아보던중에 인스타에서 많이 보여서 주문해봤어요~ 향을 고민하다가 호불호 없을것같은 아리아로 주문했는데 향도 좋은듯해요
남자친구꺼 2개주문하면서 집에도 써볼까해서 4개주문해서 거실하나 화장실에 하나 놓고 쓰는중이에요^^ 평점5 22.08.22. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

125 정말 은은한 향이 제대로 제 역할을

...

페이지 101 

페이지 102 

1622 SNS보고 주문했는데 향이 강하지 않으면서 좋습니다 평점5 22.10.24. 옵션 선택=디퓨저 리필액 1L, 향기 선택=#02_리필액 아리아 1000ml, 1+1 선택=선택 안함 

1623 발향력오지구지리구레릿고
덜어서 이방저방 나눠놈 평점5 21.09.27. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#01_디퓨저 앙쥬 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

1624 차랑집에서 쓰려고 샀어요~
향이랑 발향력이 좋아요 평점5 21.02.23. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=#02_아리아 500ml+섬유스틱(5개) 

1625 냄새도좋구발향도괜찮네요~~!!무엇보다가성비가갑!!!! 평점5 21.11.30. 옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#01_디퓨저 앙쥬 500ml, 1+1 선택=선택 안함 

1626 친구랑 쓰려고 샀는데 양도 많고 향도 좋습니다. 만족 평점5 21.11.18. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#05_디퓨저 메모리아 500ml 

1627 발향 너무잘됩니다 ㅎㅎ
근데 빨리 줄어드는건 있어용 평점5 22.08.31. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

1628 퇴근하고집에 들어오면 방안에 향이 가득해서 좋아요 평점5 21.09.14. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#01_디퓨저 앙쥬 500ml 

1629 다른 향에 비해 발향이 좀 약한데 향 자체는 괜찮아요 평점5 22.09.24. 옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#06_디퓨저 아이솝 500ml, 1+1 선택=선택 안함 

1630

1685 대용량에 냄새도 오래가서 두번째 재구매합니당💕 평점5 21.09.18. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#01_디퓨저 앙쥬 500ml, 1+1 선택=#07_디퓨저 둘치스 500m 

1686 스틱 2개만 꽂아놔도 향이 잘 퍼지네요 잘쓰겠습니다 평점5 22.05.02. 옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#03_디퓨저 벨르 500ml, 1+1 선택=선택 안함 

1687 향이 좋다고해서 구매했는데
진짜인지는 봐야겠죠?ㅈ 평점5 22.01.08. 옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

1688 향 엄청 진해서 ㅠㅠ 근데 이렇게 빨리 줄어드나요... 평점5 22.10.12. 옵션 선택=디퓨저 리필액 1L, 향기 선택=#08_리필액 북샵 1000ml, 1+1 선택=선택 안함 

1689 정말 풍자님 말되로 냄새가 온집에 나네요 ㅠ 좋아요 평점5 22.11.12. 옵션 선택: 디퓨저 향기뿜뿜 단품(54%할인) / 향기 선택: #01_디퓨저 앙쥬 500ml / 1+1 선택: 선택 안함 

1690 향이 엄청 진해서 스틱을 조금만 꽂아서 사용합니다~ 평점5 22.08.24. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

1691 크기에 놀라고 배송 빨리 와서 좋았어요
향 넘 좋아요 평점5 21.10.24. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#07_디퓨저 둘치스 500m 

1692 향이 정말 맘에 들고 차에 나두니 은은하니 좋습니다 평점5 20.08.29. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#08_아이솝 500ml+섬유스틱(5개), 1+1 선택=#02_아리아 500ml+섬유스틱(5개) 

1693 화장실에는 향이 잘 퍼지는 반면에 방은 글쎄요 ㅠㅠ 

1753 청포도향이 제일좋네요
발향은 좀더봐야할거같아요 평점5 22.03.13. 옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

1754 북샵 향기 미쳤
집 들어갈때마다 기분좋음
아 개좋아 평점5 22.06.24. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#10_디퓨저 북샵 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

1755 없으면 불안한 향기뿜뿜.....제 최애디퓨저입니당ㅋ 평점5 22.07.02. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

1756 방에 두니까 들어갈때마다 향기나서 너무 좋네요ㅎ 평점5 20.09.03. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=#06_몰리스 500ml+섬유스틱(5개) 

1757 비누향도 나고 갓 샤워한 향이 온 방에서 퍼져요 굳 평점5 21.09.20. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#08_디퓨저 아이솝 500ml 

1758 발향은 생각보다 강하지 않았지만 은은하고 좋네요 평점5 21.12.05. 옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=선택 안함 

1759 향이 진하고 오래갈것같아요 자주 이용하겠습니다! 평점5 22.06.14. 옵션 선택=디퓨저 리필액 1L, 향기 선택=#01_리필액 앙쥬 1000ml, 1+1 선택=선택 안함 

1760 향도괜찮고 용량이 대박 커요 가격대비만족입니다 평점5 20.06.22. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=#08_아이솝 500ml+섬유스틱(5개) 

1761 큰방을 다 채우네요 향기가 !!은은하니 좋아요 ㅎ헤 평점

1816 여러번 재구매 중!!진짜 가성비갑에 향도 오래 지속!! 평점5 22.08.20. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

1817 향기 조아요~~~ 엄청 강해서 집이 넘나뤼 향긋한것.... 평점5 21.11.01. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

1818 크으!!풍자언니말대로 향기굿굿 지속력너무좋아요요 평점5 21.12.25. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#06_디퓨저 몰리스 500ml 

1819 향이 잘 첨엔 너무 인위적이다 생각했는데 괜찮네요! 평점5 20.05.19. 옵션 선택=디퓨저 향기뿜뿜 단품(52%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=선택 안함 

1820 향이 정말 너무 좋아요ㅠㅠ
다시 재구매 할 것 같아용 평점5 22.08.10. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#01_디퓨저 앙쥬 500ml 

1821 양이 엄청나네요
향두 좋아요
얼마나갈진 모르겠지만 평점5 21.02.19. 옵션 선택=디퓨저 리필액 1L, 향기 선택=#05_리필액 메모리아 1000ml, 1+1 선택=선택 안함 

1822 향 진짜 뿜뿐 싸본 디퓨져중애 젤 좋아요 짱입니다용 평점5 21.10.28. 옵션 선택=디퓨저 6종세트(63%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

페이지 115 

1823 맨 윗 선반에 뒀는데도
향기가 잘 나요
청포도 반신반의하면서 구매했는데
냄새 미쳐써요 평점5 21.11.05. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

1824 

1884 화장실에 앙쥬놨는데 확실히좋아요 발향도 강하고 평점5 21.01.03. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#01_앙쥬 500ml+섬유스틱(5개), 1+1 선택=#02_아리아 500ml+섬유스틱(5개) 



In [6]:
# 5점 리뷰 데이터프레임으로 표현
df1=add_dataframe(name_,category_,reviews,stars,dates,options,cnt)
df1.head()

,type,category,review,star,date,option
1,헤트라스,별점,"제가 캔들,디퓨저 만드는 사람입니다~\n디퓨져는 사서는 써본적없고 받아서만 써보았어...",평점5,21.06.24.,"옵션 선택=디퓨저 리필액 1L, 향기 선택=#02_리필액 아리아 1L, 1+1 선택..."
2,헤트라스,별점,디퓨저 좋아하는 사람이라 이것저것 많이 옴겨타다가 인스광고보고 혹시나하고 사봤어요....,평점3,22.07.19.,"옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향..."
3,헤트라스,별점,가격도 저렴하고\n작은 통에 소분해서 여러군데 놔뒀는데\n일단 좁은 곳에서는 발향이...,평점5,22.08.12.,"옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#02_디퓨저 아리아 50..."
4,헤트라스,별점,처음 구매하는거라 향이랑 발향력 볼려구 단품주문햇는데.. 박스에서 이미 디퓨저향이 ...,평점5,22.10.19.,"옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#02_디퓨저 아리아 50..."
5,헤트라스,별점,풍자님이 알려주셔서 2주전에 구매했는데\n\n진짜 거짓말이 아니고 다른 유명하다는 ...,평점5,21.11.29.,"옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향..."


In [7]:
# 크롤링한 5점 데이터의 갯수
len(df1)

1884

In [8]:
# 5점 데이터 전처리

df1 = df1.drop_duplicates()
df1 = df1.reset_index(drop=True)
df1_1 = df1['option'].str.split('/',expand=True)
df1 = pd.concat([df1,df1_1],axis=1)
df1.head()

,type,category,review,star,date,option,0,1,2,3
0,헤트라스,별점,"제가 캔들,디퓨저 만드는 사람입니다~\n디퓨져는 사서는 써본적없고 받아서만 써보았어...",평점5,21.06.24.,"옵션 선택=디퓨저 리필액 1L, 향기 선택=#02_리필액 아리아 1L, 1+1 선택...","옵션 선택=디퓨저 리필액 1L, 향기 선택=#02_리필액 아리아 1L, 1+1 선택...",None,None,None
1,헤트라스,별점,디퓨저 좋아하는 사람이라 이것저것 많이 옴겨타다가 인스광고보고 혹시나하고 사봤어요....,평점3,22.07.19.,"옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향...","옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향...",None,None,None
2,헤트라스,별점,가격도 저렴하고\n작은 통에 소분해서 여러군데 놔뒀는데\n일단 좁은 곳에서는 발향이...,평점5,22.08.12.,"옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#02_디퓨저 아리아 50...","옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#02_디퓨저 아리아 50...",None,None,None
3,헤트라스,별점,처음 구매하는거라 향이랑 발향력 볼려구 단품주문햇는데.. 박스에서 이미 디퓨저향이 ...,평점5,22.10.19.,"옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#02_디퓨저 아리아 50...","옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#02_디퓨저 아리아 50...",None,None,None
4,헤트라스,별점,풍자님이 알려주셔서 2주전에 구매했는데\n\n진짜 거짓말이 아니고 다른 유명하다는 ...,평점5,21.11.29.,"옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향...","옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향...",None,None,None


In [9]:
# 전처리 후 크롤링한 5점 데이터의 갯수
len(df1)

1838

In [10]:
# 웹페이지 열기

header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_13888/2638303677.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [11]:
# 4점 리뷰 가져오기
d.find_element(By.XPATH,category2).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
dates=[]
options=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    #sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            date=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[4]').text
            dates.append(date)
            option=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[5]').text
            options.append(option)
            #if j%2==0: #화면에 2개씩 보이도록 스크롤
            #    ELEMENT = d.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
            #    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review, star, date, option, "\n")
            cnt+=1 
        except: break
            
        #sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
        
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[12]').click()
            else : next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
#save()

페이지 1 

1 제가 캔들,디퓨저 만드는 사람입니다~
디퓨져는 사서는 써본적없고 받아서만 써보았어요,이번에 한번 사서 쓰게 되어보았어요. 제가 만들어 쓰는향들은 하두 익숙해져서 향이 엄청 강한향을 좋아라하니..색다른 향은 냄새를 금방 느끼는데 늘 맡던 향엔 코가 적응 하였는지 많이 안나는거같더라구요. 저에겐 없는 청포도 향을 사보았어요. 글고 빨래냄새~ 빨래냄새 일반 제가 쓰는 용액 다우니보단 향은 덜해요. (저는 향을 하도 많이 맡아서그런데 보통사람들에겐 진할수도있어요) 청포도향은 처음 맡는 향이라 진하게 느껴지고 실제로도 다른향보단 강한거같제요. 회사에서 이층에 청포도 해놓으니 1층까지도 발향될정도로 좋아요~ 제 제품과 조금 다른게 있다면 용액 닳는 속도 가 조금 빠른거같기두해요~그래도 발향이 잘되서 좋네여 평점5 22.10.11. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

2 아리아와 아이솝 구매했습니다
다우니 향을 엄청 기대했는데! 다우니 에어프릴향이 전혀 아닌거 같아요! 되게 달달한 냄새가 나네요! 용량은 커서 오래 사용할것 같아요 사용한지는 3주 정도 된것 같은데! 향이 넓게 퍼지는 편같아요!
가까이 있으면 너무 달아서 코랑 머리가 아파서 멀리 두고 사용을 추천합니다.
시원하고 포근함 섬유유연제 향이 안나서 아쉬워요 엄청 기대했는데요욥
비누향은 아직 개봉은 안했는데 도브 보다는 그냥 비누향 같은 향인데 너무 진하지 않아서 괜찮을거 같아요! ! 평점4 21.01.14. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=#08_아이솝 500ml+섬유스틱(5개) 

3 발향력 솔직히 좋다길레 보통보다 살짝 좋은정도겠지 했는데 상상 이상이네요 40평저희집에 제방에 두어도(평소에 문 거의 닫고생활) 안방까지 냄새가 가요 제 방 위치가 현관 문앞인걸 생각했을때 정말 발향하나는 최고입니다 대신 조금 머리가 띵하네요 막대?

33 향은 두 가지 다른걸로 선택해서 구매했고
이건 다우니향인데
리뷰에 다우니향 두배로 난다고 냄새 좋다고
극찬 극찬을 하길래 다우니향 너무 기대하면서 구매했는데 생각했던거랑은 향이 별로에요
너무 진한거 같기도하고 ㅠ 용량도 커서
저 많은 향이 날라갈때까지 써야하니.........
다음번엔 또 다른 향으로 구매해서 젤 좋은게 뭔지 알아보려고합니다! 평점4 22.01.24. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#03_디퓨저 벨르 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

34 가득 채웠는데 3일만에 반이 줄었어요. 광고하는 것만큼 발향이 매우 우수하진 않은데 진짜 잘 날아가네요. 금방 닳아요. 그럼에도 계속 구입하는 이유는 방향제 중엔 향 맡으면 머리 지끈 거리고 아픈 경우도 있던데 얜 그렇진 않아서요. 평점4 22.08.02. 옵션 선택=디퓨저 리필액 1L, 향기 선택=#07_리필액 가든파티 1000ml, 1+1 선택=선택 안함 

35 화장실 선반에 올려놨어요
청포도향 구매했는데 처음엔 생각보다 알코올 냄새같은게 좀 많이 나서 이게 청포도향이라고? 했는데
며칠 지나니까 찐 청포도향기가 났어요
다들 입모아 발향이 최고라고 하는데 저는 그부분에선 다른 제품이랑 그렇게 차이나는지 잘 모르겠어요ㅋㅋ
용량이랑 다른 것들 따졌을 때 금액대에 비해 좋은 제품이라 생각해요. 만족도는 높습니다 ^^ 평점4 22.03.05. 옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#03_디퓨저 벨르 500ml, 1+1 선택=선택 안함 

36 한달조금넘게 사용했네요
좋아요 근데 향이 너무쎄서 바깥까지 냄새가 나가요 ㅎㅎㅎ
대용량이라 향이 금방질려서 두번은 안살거같아요
그래도 좋습니다 집환기안시키고 디퓨저만두면
집들어왔을때 향이쎄서 향이랑 집냄새랑 합쳐져서 약간 꿉꿉? 눅눅한냄새 라고해야되나 그런게나요 평점4 20.04.22. 옵션 선택=디퓨저 BEST3(65%할인), 향기 선택=BEST3세트(앙쥬+아리아+벨르), 1+1 선택=선택 안

70 상큼한 포도향보다는
오래된 진한 청포도향이에요
색상이 무색인줄 알았는데 노란색이네요
양은 많아서 오래쓸듯해요
향이 진하긴하지만 무해하다고하니
안심하고 사용해볼께요 평점4 22.03.20. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#03_디퓨저 벨르 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

71 배송일주일걸렷네여 오자마자 다섯개스틱 다꽂앗는데 후기처럼 온집이 다퍼지는진 잘모르겟네용 근처에가면 향나요 평점4 21.07.22. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

72 생각하는거 만큼 향이 진하게 퍼지진 읺는데 디퓨져 있는곳 지나가면 향기가 나요
병은 공병에 옮겨서 사용중이예요 평점4 21.11.17. 옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#08_디퓨저 아이솝 500ml, 1+1 선택=선택 안함 

73 앙쥬와 벨르 구매했는데
달달한 향이긴
하지만 둘 다 첫향에서 알콜 냄새가 많이 올라왔어요
향이 강하고 오래가기는 하는데 멀리까지 퍼지지 않는 것 같아요 평점4 21.08.11. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#01_디퓨저 앙쥬 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

74 아이솝 향이 마음에 들어서 재구매 하면서 비쥬도 구매해봣어요 상큼하니 좋아요 아이솝이나 비쥬 둘다 향이 엄청 강하진 않은 것 같아요
공간을 꽉 채워주지 못하고 근처에 가거나 바람이 불어야 나는거 같아서 아쉽네요 평점4 21.06.03. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#08_아이솝 500ml+섬유스틱(5개), 1+1 선택=#04_비쥬 500ml+섬유스틱(5개) 

75 담엔 다우니 향을 사보려구요~^^
향이 은은하게 퍼지긴 하는데 광고처럼 넓은 곳을 다 채우기엔 부족한 것 같아요~! 평점4 22.01.30. 옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택

페이지 8 

121 배송은 쏘쏘 하고 아직까지는 써보지는 않았지만 향이 자연스럽게 날거라고 기대가되네요^^~근데 물이 원래 누런건가요? 한개는 물이 하얗고 나머지한개는 누래요🤔 뭐죠? 평점4 22.02.24. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#01_디퓨저 앙쥬 500ml, 1+1 선택=#07_디퓨저 둘치스 500m 

122 향기는 생각했던거보단 덜한느낌인데
발향력?은 좋습니다
차량컵홀더에 딱맞는사이즈여서 만족합니다.
차에서도 좋은향기가 나네요 평점4 22.08.19. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#04_디퓨저 몰리스 500ml 

123 고급진 향기는아님
케이스에 붙어있는 스티커가 향과 어우러져 더 싼티남..
발향력은 우수함
총평: 가성비좋은데 케이스는 커버가안됨... 평점4 22.07.06. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

124 향은 좋은데 포장이 엉망이에요 하나 샜네요 리뷰보고 설마 나는 아니겠지 했는데 흠… 평점4 22.10.25. 옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

125 유투버 광고 보고 구매했는데 향이 막 퍼지는건 아니고 그냥 옆에 지나칠때 살짝 나는 정도 평점4 21.12.03. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#01_디퓨저 앙쥬 500ml, 1+1 선택=#01_디퓨저 앙쥬 500ml 

126 향기가 살짝 아쉬운데 좀 더 진하게 발향되면 좋겠어요ㅠㅠ우선 쟁여놨으니 계속 써볼게요 평점4 21.06.02. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#01_앙쥬 500ml+섬유스틱(5개), 1+1 선택=#01_앙쥬 500ml+섬유스틱(5개) 

127 오 냄새 진짜잘퍼져요 용량도갑이구용담에다른향사봐야겠어요 설마하구

179 전보다 향이 많이 약해졌어요...
향이 오래간다고해서 샀는데 이번엔 잘 안나네요 ㅠㅠ
암튼 향은 좋습니다~ 평점4 22.04.12. 옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

180 다우니향이라해서 삿는데 다우니향은 아닌것같아요.. 발향력은 좋아요 하나만 꽂아도 널리널리 퍼져나갑니다. 전 괜찮은데 남편은 향기가 머리아프다하네요ㅜㅜ 평점4 22.05.20. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

181 향이 많이 난다고 하나만 꼽는다던데 저는 두개 꽂아도 별로안나네요 ㅠㅠ 평점4 20.07.05. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=#06_몰리스 500ml+섬유스틱(5개) 

페이지 12 

182 배송도빠르고 확실그전썻던것보단발향이 좋은듯해요. 다른향도써보고싶네요~ 평점4 22.05.16. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

183 처음 개봉할때는 발향이 잘돼서 좋은데 시간지나면 무뎌지는건 어쩔수 없나봐요 그럴때마다 막대기를 뒤집어주면 다시 향기가 나긴해요 그리고 항상 마지막에 조금 남은게 안없어지고 계속 있더라구요,, 평점4 22.10.14. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#06_디퓨저 아이솝 500ml, 1+1 선택=#06_디퓨저 아이솝 500ml 

184 배송빠르고 발향?은 다른디퓨저보다 괜찮은거같은데 향이쏘쏘...후각이 엄청예민한사람으로써 어지럽거나 독하진않지만 저렴한향기..? 남자화장품 스킨냄새...? 종류별로 향써보고 정착해야할듯요 평점4 21.08.11. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#06_디퓨저 몰리스 500ml, 1+1 선택=#06_디퓨저 몰

...

637 좋아요 완전 만족 향기 굳 평점4 21.01.26. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#08_아이솝 500ml+섬유스틱(5개), 1+1 선택=#02_아리아 500ml+섬유스틱(5개) 

638 향이좋아서잘쓰고있어요? 평점4 22.11.22. 옵션 선택: 디퓨저 향기뿜뿜 1+1(55%할인) / 향기 선택: #03_디퓨저 벨르 500ml / 1+1 선택: #03_디퓨저 벨르 500ml 

639 향진해요 오래갈듯하네요 평점4 21.09.21. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

640 발향좋아요 재구매했어요 평점4 22.01.10. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#08_디퓨저 아이솝 500ml 

641 배송빠르고 포장꼼꼼해요 평점4 22.07.27. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#03_디퓨저 벨르 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

642 아직 개봉은 안했지만 냄새는 좋아요, 강한냄새보다는 은은하게 퍼질정도의 향인것같아요. 평점4 20.05.19. 옵션 선택=디퓨저 BEST3(65%할인), 향기 선택=BEST3세트(앙쥬+아리아+벨르), 1+1 선택=선택 안함 

643 향은 생각보다 연하네여 방이 조그만해서 처음에 2개꼿고 냄새 안나서 3개 했는데 은은하게 향이 나네여
향은 혜자인데 생각보다 빨리 달아여 평점4 22.02.12. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

644 향기도 좋고 너무 좋아요 평점4 20.09.02. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=#04_비쥬 500ml+섬유스틱(5개) 

645 향이 계속 정말

713 냄새굿 향이 오래가요 평점4 22.07.08. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#01_디퓨저 앙쥬 500ml, 1+1 선택=#01_디퓨저 앙쥬 500ml 

714 향이 좋고 멀리퍼쟈요 평점4 22.06.29. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

715 냄새가 발향이 좋네요 평점4 22.08.27. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#06_디퓨저 아이솝 500ml 

716 가성비 아주좋아요 ㅋ 평점4 20.08.19. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=#03_벨르 500ml+섬유스틱(5개) 

717 양이 많아서 좋습니다 평점4 22.10.25. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

718 좋다고해서 사봤어요~ 평점4 22.07.25. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#07_디퓨저 북샵 500ml 

719 짐승용량이라 걱정없네요
근데 진짜 두개만 꽂았는데
발향력 좋네요 평점4 22.09.07. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#07_디퓨저 가든파티 500ml, 1+1 선택=#04_디퓨저 몰리스 500ml 

720 생각만큼 냄새가 온방안에 퍼지지는 않는데요?
그래도 냄새는 좋네요 평점4 22.08.22. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

페이지 45 

721 양도많고 향도 좋아요 평점4 20.10.22. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=#03_벨

785 막광고처럼 발향이 좋은건 아니지만 용량크고 향이 맘에 들어요 평점4 22.02.26. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#09_디퓨저 가든파티 500ml, 1+1 선택=#09_디퓨저 가든파티 500ml 

786 향좋아요 향도 잘퍼지는것 같아요
양이 많이서 아주 맘에 들어요 평점4 21.06.17. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#08_디퓨저 아이솝 500ml 

787 은은하네요 평점4 21.01.29. 향선택 / #03_벨르500ml / 9,500원 / 

788 저렴해서 쓰긴하는데 그리 향이 널리 퍼지진않아요 어떨땐 향이나다 안나다 그래요ㅠ 평점4 22.08.21. 옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

789 향도 강하고 좋아요
배송도 빠르고
용량이 커서 넘 좋은거 같아요 평점4 22.02.25. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#07_디퓨저 둘치스 500ml, 1+1 선택=#01_디퓨저 앙쥬 500ml 

페이지 50 

790 생각보다 확 냄새가 퍼지지 않는것 같아요.ㅠ좀더 지켜봐야겠어요. 그래도 양과 향은 좋아요. 비누향 좋아요 평점4 22.06.02. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

791 배송이 10일 걸려서 별하나 빼요
청포도 향이 발향이 제일 잘되는것같아서 다음엔 청포도만 사려고요 평점4 22.03.22. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

792 향은너무너무좋고 용량도큰데 빠른속도로 줄어드네요 평점4 20.10.04. 옵션 선택=디퓨저 BEST3(65%할인), 향기 선택=BEST3세트(앙쥬+아리아+벨르), 1+1 선택=선택 안함 

793 택배회사의 오류로 배

846 향기가 대박이에요 방이랑 화장실이랑 향기롭게 지내고있습니다 평점4 21.09.05. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

847 발향이 후기만큼 좋지는 않아요 6평에서 사용중인데 향이 전체적으로 퍼지진 않아요..! 평점4 22.07.22. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#08_디퓨저 북샵 500ml 

848 향 좋아요!
근데 은은하고 고급향은 아닌거 같아요.ㅎㅎ
그치만 양도 많고!!!
가성비 넘 좋아요!! 평점4 22.05.10. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#09_디퓨저 가든파티 500ml 

849 향이 너무좋아요!
오자마자 엄마가 하나 가져가셧네요 ㅋㅋㅋ
대용량이라 더 좋아요! 평점4 21.06.06. 옵션 선택=디퓨저 BEST3(60%할인), 향기 선택=디퓨저 BEST3(배송메세지에 향 작성), 1+1 선택=선택 안함 

페이지 53 

850 배송은 오배송1건 있었지만 대처나 여러부분에서는 신속하게 하려 노력해주셨습니다, 급한상황이었다면 안좋았겠지만 나름 만족합니다. 평점4 21.03.13. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=#01_앙쥬 500ml+섬유스틱(5개) 

851 향도 좋고 발향도 좋은데 너무 빨리 증발하네요 평점4 22.10.18. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

852 2차 구매입니다. 감사합니다. 수고하세요! 평점4 20.02.05. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#01_앙쥬 500ml+섬유스틱(5개), 1+1 선택=#01_앙쥬 500ml+섬유스틱(5개) 

85

911 향좋네요 또구매할걱ㅇㄴㆍㄱ 평점4 22.01.23. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

912 발향력만은 인정합니다 만족합니다 평점4 22.10.28. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

913 다우니향이라서 삿는데 전혀 다우니향이 아니에요… 평점4 22.09.25. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

914 배송도 빠르고 ㅈㅎㅎ아요 평점4 22.11.16. 옵션 선택: 디퓨저 리필액 1L / 향기 선택: #07_리필액 가든파티 1000ml / 1+1 선택: 선택 안함 

915 하나는 침실에 하나는 욕실에 뒀어요~ 양도 많고 애용할듯 평점4 21.05.20. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=#08_아이솝 500ml+섬유스틱(5개) 

916 좋아요 또주문해요 리필도 있고 좋아요 평점4 22.11.27. 옵션 선택: 디퓨저 향기뿜뿜 1+1(55%할인) / 향기 선택: #10_디퓨저 플로랑 500ml / 1+1 선택: #10_디퓨저 플로랑 500ml 

917 청포도향은 사탕청포도향이에욥 배송빨라요 평점4 22.10.18. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#03_디퓨저 벨르 500ml, 1+1 선택=#07_디퓨저 가든파티 500ml 

918 너무 너무 상쾌합니닿 평점4 21.09.10. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#03_디퓨저 벨르 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

919 발향 너무너무 좋아요 평점4 22.03.30. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#01_디퓨저 앙쥬 50

In [12]:
# 4점 리뷰 데이터프레임으로 표현
df2=add_dataframe(name_,category_,reviews,stars,dates,options,cnt)
df2.head()

,type,category,review,star,date,option
1,헤트라스,별점,"제가 캔들,디퓨저 만드는 사람입니다~\n디퓨져는 사서는 써본적없고 받아서만 써보았어...",평점5,22.10.11.,"옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향..."
2,헤트라스,별점,아리아와 아이솝 구매했습니다\n다우니 향을 엄청 기대했는데! 다우니 에어프릴향이 전...,평점4,21.01.14.,"옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#02_아리아 500ml..."
3,헤트라스,별점,발향력 솔직히 좋다길레 보통보다 살짝 좋은정도겠지 했는데 상상 이상이네요 40평저희...,평점4,21.11.28.,"옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#06_디퓨저 몰리스 50..."
4,헤트라스,별점,입구쪽에 고무 마개 때문에 애먹었어요…빼는 건지 그대로 뚫어서 쓰는건지 도통 알수없...,평점4,21.10.25.,"옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#02_디퓨저 아리아 50..."
5,헤트라스,별점,"후각에 민감한편이라 여기저기 ㄷㅣ퓨저, 향초 등 많이 구매도하고 사용도 해보았는데 ...",평점4,21.12.07.,"옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향..."


In [13]:
# 크롤링한 4점 데이터의 갯수
len(df2)

946

In [14]:
# 4점 데이터 전처리

df2 = df2.drop_duplicates()
df2 = df2.reset_index(drop=True)
df2_2 = df2['option'].str.split('/',expand=True)
df2 = pd.concat([df2,df2_2],axis=1)
df2.head()

,type,category,review,star,date,option,0,1,2,3
0,헤트라스,별점,"제가 캔들,디퓨저 만드는 사람입니다~\n디퓨져는 사서는 써본적없고 받아서만 써보았어...",평점5,22.10.11.,"옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향...","옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향...",None,None,None
1,헤트라스,별점,아리아와 아이솝 구매했습니다\n다우니 향을 엄청 기대했는데! 다우니 에어프릴향이 전...,평점4,21.01.14.,"옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#02_아리아 500ml...","옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#02_아리아 500ml...",None,None,None
2,헤트라스,별점,발향력 솔직히 좋다길레 보통보다 살짝 좋은정도겠지 했는데 상상 이상이네요 40평저희...,평점4,21.11.28.,"옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#06_디퓨저 몰리스 50...","옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#06_디퓨저 몰리스 50...",None,None,None
3,헤트라스,별점,입구쪽에 고무 마개 때문에 애먹었어요…빼는 건지 그대로 뚫어서 쓰는건지 도통 알수없...,평점4,21.10.25.,"옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#02_디퓨저 아리아 50...","옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#02_디퓨저 아리아 50...",None,None,None
4,헤트라스,별점,"후각에 민감한편이라 여기저기 ㄷㅣ퓨저, 향초 등 많이 구매도하고 사용도 해보았는데 ...",평점4,21.12.07.,"옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향...","옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향...",None,None,None


In [15]:
# 전처리 후 크롤링한 4점 데이터의 갯수
len(df2)

924

In [16]:
# 웹페이지 열기

header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_13888/2638303677.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [17]:
# 3점 리뷰 가져오기
d.find_element(By.XPATH,category3).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
dates=[]
options=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    #sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            date=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[4]').text
            dates.append(date)
            option=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[5]').text
            options.append(option)
            #if j%2==0: #화면에 2개씩 보이도록 스크롤
            #    ELEMENT = d.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
            #    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review, star, date, option, "\n")
            cnt+=1 
        except: break
            
        #sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
        
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[12]').click()
            else : next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
#save()

페이지 1 

페이지 2 

1 디퓨저 좋아하는 사람이라 이것저것 많이 옴겨타다가 인스광고보고 혹시나하고 사봤어요.
발향이 좋은지 안좋은지 사실 알송달송해요;;
6개세트 청포도 구매했구요 거실2개
신발장현관1개 중문지나서1개 화장실한개씩 한번에 다 설치했어요.
거실은 향없구요
나름 밀폐된곳 화장실 현관은
스틱 4개씩 꼽아뒀구요.
중문입구1개 식탁위1개 거실장1개 이렇게 나눴는데 디퓨저에 코대봐도 그다지 안나요
그렇다고 향이 없나하기엔 화장실 현관에는 향이 나구요... 강하게나고 그러진 않아요 거실에 향이 퍼지길 바랫는데 씁쓸 34평 일반 아파트입니다...
가격저렴하니 그냥 그러려니하는데 재구매는 안할꺼같아요4
디퓨저 육목민이라... 평점3 22.07.19. 옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

2 일단..
저한테는 좀 안맞는거같아요
향이 너무 진하게 퍼지다보니 머리 아파서,
처음엔 2개 꼽았다가 20분도 안대서 한개빼고, 그래도 향이 강해서 입구부분을 테이프로 막아놨어요;;;
최대한 공기중에 노출되지않게..
문 열어서 환기도 겁나 시키고ㅋ
음식을 먹어도 입맛이 사라짐🥲 음식냄새를 누르고 향기를 뿜뿜해버리네요
진하게 지속되는걸 원하시는분들이면 만족하실거예요.
근데 저처럼 은은하게 나는걸 원한다면 비추합니다 평점3 21.11.11. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#01_디퓨저 앙쥬 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

3 스틱 2개꼽아도 향이 가득하다던데 첫날빼고는 그닥 효과를 못본거같아요 다른한개는 3개했다 거실이라 하루뒤 5개꼽았는데도 아쉽더라고여
제가 좋아하는 향의 느낌이 아니라서 다 떨어지면 다른향으로 재구매할 의향은 있어요 다른 향으로 실험 해보고 싶네요~ 양이 많아서 좋긴한데 디자인적으론 살작 아쉬워요ㅠㅋㅋ손님들 왔을때 뭔가 보여주기 창피한 느낌이 없잖아 있네여~ 평점3 22.02.14. 옵션 선택=디퓨저 향기뿜뿜 1+1(

35 향이 잘퍼진다고 해서 샀는데, 음...원룸인데도 만족스럽진 않네요. 향기는 좋습니다.
그 근처에 문을 여닫거나 하면 향이 퍼지는 정도.. 정말 좁은 원룸인데 집 들어가자마자 향이 풍기고 이런건 아니더라구요. 그냥 그근처.. 평점3 22.06.17. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#09_디퓨저 가든파티 500ml, 1+1 선택=#10_디퓨저 북샵 500ml 

36 일단 양은 정말 대혜자,, 완전 용량 큼지막합니다.
리뷰엔 하나만 꽂아도 향이 너무 심하다고 하던데 전 세개 꽂아도.. 방이 좁은데도 그닥 냄새 많이 안났어요ㅠㅠ
앙쥬랑 벨르 샀는데 앙쥬가 더 냄새 많이 나는거 가타여 평점3 20.11.06. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#01_앙쥬 500ml+섬유스틱(5개), 1+1 선택=#03_벨르 500ml+섬유스틱(5개) 

37 리뷰가 너무 좋아서 기대를 너무 많이 한것같요 ^^;

생각했던 그런 향들이 아니라서 조금 아쉽네요.
그래도 이 가격대 1+1으로 잘산거같아 잘 사용하겠습니다. 평점3 21.08.20. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

38 그냥 무난해요. 솔직히 광고만큼 엄청 발향력이 좋지는 않아요. 향은 좋은데...
과대광고만 아니었어도 만족하면서 썼을텐데 본 게 있어서 기대를 하고 썼더니 만족도가 그닥이네요. 평점3 22.11.07. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#06_디퓨저 아이솝 500ml 

39 짐승용량은 맞으나.. 첨 열었을때 알콜향이 심합니다. 그냥 가격대비 용량이 많은정도입니다. 냄새때문에 머리아파서 방에는 못둘것같아요. 통은 스티커제거후 리폼해봤습니다 평점3 20.01.08. 옵션 선택=디퓨저 BEST3(65%할인), 향기 선택=BEST3세트(앙쥬+아리아+벨르), 1+1 선택=선택 안함 

86 향이 색각보다 달아서 100프로 만족하진 않아요! 그래도 용량이 커서 좋네요 평점3 21.07.26. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

87 음...재구매라 대용량리필 구매했는데 리필용이라 그런건지 향이 별로 안나요....스틱을 더 꽂으니 양이 눈에 띄게 확 줄어들구요 평점3 22.07.01. 옵션 선택=디퓨저 리필액 1L, 향기 선택=#03_리필액 벨르 1000ml, 1+1 선택=선택 안함 

88 빠른배송 만족, 허나 향기뿜뿜인데 발향이 그렇게 쎈거같진않네요 북샵향인데 서점에서 맡아봤던 향이라 하기엔 약함니다. 평점3 22.08.12. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#08_디퓨저 북샵 500ml, 1+1 선택=#08_디퓨저 북샵 500ml 

89 발향이좋다던데 전못느끼겟더라고용
예민보스인데 기대를 너무햇나봐여ㅋ 평점3 22.03.02. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#03_디퓨저 벨르 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

90 발향이 넢게된다거해서 구매햇는데 한달꺼본결과 그렇게 막 좋진않아요ㅠ 평점3 22.06.14. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#08_디퓨저 아이솝 500ml 

91 인스타 광고가 계속 뜨길래
몬가싶어 도대체 궁금해서 사봄
쏘쏘. . 좋아요 평점3 21.12.25. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#05_디퓨저 메모리아 500ml, 1+1 선택=#08_디퓨저 아이솝 500ml 

92 처음에 향이 강해서 부담스러웠는데 점점 은은하게 바뀌더니 너무좋네요 평점3 19.12.18. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=#02_아리아 500ml+섬유스틱(5개) 

93 리뷰믿고

148 디퓨저하나는1/3이 쏟아져서왔습니다.
발향력은좋아요 평점3 21.11.29. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

149 한달 지났는데 향은 많이 날아가서 잘 안납니다. ^^;;; 평점3 21.12.20. 옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#04_디퓨저 비쥬 500ml, 1+1 선택=선택 안함 

페이지 12 

페이지 13 

150 2개샀어요 ! 방에둔건 은은히 향 잘나는데
거실에는 향이 넓게 퍼지진않네요
있는곳 지나다닐때만 나요;;
그랴도 양많아서 좋음! 평점3 21.09.08. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#07_디퓨저 둘치스 500m 

151 생각보다 발향이 뿜쁨하지않아서 기대이하였어요... 향마다 차이가 있는건지모르겠지만... ㅠㅠ 향이.잘 퍼질줄알았는데 아님... 평점3 22.01.10. 옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#10_디퓨저 북샵 500ml, 1+1 선택=선택 안함 

152 배송도 빠르고 향은좋은데 어쩐지 기우뚱거리더라니..한쪽이 저렇게 구겨져서 왓어요ㅠㅜㅠ 이게뭐람.. 평점3 20.08.11. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=#04_비쥬 500ml+섬유스틱(5개) 

153 일단 양은 많아요 받자 마자 스틱 꽃아두니 일단 향은 잘 퍼지는듯해요 이게 근데 처음 새거라 그런지는 모르겠지만 두고두고 오랜시간 써봐야 알것같네요 일단 용량은 인정👍 평점3 21.12.09. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#05_디퓨저 메모리아 500ml 

154 왜우리집은...향기가 안나죠? 쩝
가까이에서 맡으면 나는데 밖에나갔다 들어와도 확 꽂히게 나진않음... 평점3 22.04.04. 옵션

203 잘못 포장한건지 디퓨저 용액이 새서 왔네요..... 근데 향은 좋아요 평점3 22.10.13. 옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#01_디퓨저 앙쥬 500ml, 1+1 선택=선택 안함 

204 향이많이난다는데..글쎄요 아직은잘모르겟네요 참고로하루됏어요 평점3 20.07.05. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#01_앙쥬 500ml+섬유스틱(5개), 1+1 선택=#05_메모리아 500ml+섬유스틱(5개) 

205 다 좋은데 조금씩 새서 배송이 됐네요
포장을 잘 해야 할꺼 같아요! 평점3 22.05.12. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#06_디퓨저 몰리스 500ml, 1+1 선택=#09_디퓨저 가든파티 500ml 

206 25평인데 2통을 놔뒀는데 향이 은은하게 나긴 하네요~ 이제 보름 되었는데 반통 넘게 날아갔어요 ㅎ 평점3 22.05.15. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#09_디퓨저 가든파티 500ml, 1+1 선택=#10_디퓨저 북샵 500ml 

207 향은생각보다는 확나진 않은듯하네요 옷방에 뒀는데 은은합니다 평점3 22.01.10. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#08_디퓨저 아이솝 500ml 

208 방이 작은대도 향이 가득찬 느낌은 안나요
음.. 대용량이기만한듯 평점3 22.04.10. 옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

209 일단 배송확인도지대로안하고 분실되서 2주만에받음 향은 쏘쏘 평점3 22.02.12. 옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#08_디퓨저 아이솝 500ml, 1+1 선택=선택 안함 

페이지 18 

210 생각보다 냄새가 많이 나진않는듯 합니다.
그래도 가성비 좋아요 평점3 21.11.14. 옵션 선택=섬유스틱(5개세트), 향

273 향이 오래 가진 않네요 ㅠ 평점3 22.01.16. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#01_디퓨저 앙쥬 500ml, 1+1 선택=#05_디퓨저 메모리아 500ml 

274 향기좋아요. 만족합니다. 평점3 22.02.11. 옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#06_디퓨저 몰리스 500ml, 1+1 선택=선택 안함 

275 향이 강하진않읒것같아요 평점3 21.08.03. 옵션 선택=디퓨저 6종세트(63%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

276 청포도향은 별로에여ㅠㅠ 평점3 22.10.21. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

277 무난하게 사용한것같아요 평점3 22.04.06. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

페이지 23 

278 방에 두었는데 향이 약해요 평점3 21.04.16. 옵션 선택=디퓨저 향기뿜뿜 단품(52%할인), 향기 선택=#01_앙쥬 500ml+섬유스틱(5개), 1+1 선택=선택 안함 

279 가성비가 좋아오아라어이 평점3 20.11.25. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#01_앙쥬 500ml+섬유스틱(5개), 1+1 선택=#01_앙쥬 500ml+섬유스틱(5개) 

280 다른 병에 담아서 쓰는 중 평점3 21.10.25. 옵션 선택=디퓨저 리필액 1L, 향기 선택=#05_리필액 메모리아 1000ml, 1+1 선택=선택 안함 

281 조금 세서 왔어요 젖어서 평점3 22.08.20. 옵션 선택=디퓨저 리필액 1L, 향기 선택=#08_리필액 북샵 1000ml, 1+1 선택=선택 안함 

282 발향 좋은만큼 빨리 닳음 평점3 22.03.20. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향

344 향은 좋은거같은데 잘안퍼지는거같음 평점3 20.10.28. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#04_비쥬 500ml+섬유스틱(5개), 1+1 선택=#03_벨르 500ml+섬유스틱(5개) 

345 향이 좋은데 잘안퍼지네요 집안에서 샌각보다 향이 안퍼집니다 평점3 21.02.05. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#06_몰리스 500ml+섬유스틱(5개), 1+1 선택=#08_아이솝 500ml+섬유스틱(5개) 

346 생각보다 향이 퍼지지않네요
향이 가까이 가야 나서 실망이네요 평점3 22.02.21. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#06_디퓨저 몰리스 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

347 배송이 엄청 느렸어요 향은 진짜 진하게 잘 나네요 재구매 의사 있지만 배송에 신경써주세요 평점3 22.03.11. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

페이지 29 

348 향은 정말 좋아요 강한 향좋아하는나에게는 약해요ㅜ 평점3 22.11.09. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#01_디퓨저 앙쥬 500ml, 1+1 선택=#01_디퓨저 앙쥬 500ml 

349 향이 강하지는 않아요 ㅋㅋ 용량은 대용량 평점3 20.06.14. 옵션 선택=디퓨저 향기뿜뿜 1+1(62%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=#02_아리아 500ml+섬유스틱(5개) 

350 대용량이라 좋긴한데, 향이 지속적으로 강한느낌은 없네요ㅜㅜ 지금 절반 이상 사용했는데 사용된건 많이 보이지만 향이 별로 안나요ㅜㅜ 평점3 22.07.22. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#08_디퓨저 아이솝 500ml, 1+1 선택=#08_디퓨저 아이솝 500ml 

351 택배파업땜인지 배송은 늦었지만 향은 잘나

In [18]:
# 3점 리뷰 데이터프레임으로 표현
df3=add_dataframe(name_,category_,reviews,stars,dates,options,cnt)
df3.head()

,type,category,review,star,date,option
1,헤트라스,별점,디퓨저 좋아하는 사람이라 이것저것 많이 옴겨타다가 인스광고보고 혹시나하고 사봤어요....,평점3,22.07.19.,"옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향..."
2,헤트라스,별점,"일단..\n저한테는 좀 안맞는거같아요\n향이 너무 진하게 퍼지다보니 머리 아파서,\...",평점3,21.11.11.,"옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#01_디퓨저 앙쥬 50..."
3,헤트라스,별점,스틱 2개꼽아도 향이 가득하다던데 첫날빼고는 그닥 효과를 못본거같아요 다른한개는 3...,평점3,22.02.14.,"옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#05_디퓨저 메모리아 ..."
4,헤트라스,별점,생각보다 지속력이 좋진 않은 듯\n초반 발향력은 매우 좋았음 그건 정말 ㅇㅈ\n퇴근...,평점3,21.09.07.,"옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 5..."
5,헤트라스,별점,진짜 잘 닳아서 대용량으로 구매했어요. 발향이 항상 잘 되는 건 아니에요. 스틱 교...,평점3,22.08.02.,"옵션 선택=디퓨저 리필액 1L, 향기 선택=#08_리필액 북샵 1000ml, 1+1..."


In [19]:
# 전처리 후 크롤링한 3점 데이터의 갯수
len(df3)

377

In [20]:
# 3점 데이터 전처리

df3 = df3.drop_duplicates()
df3 = df3.reset_index(drop=True)
df3_3 = df3['option'].str.split('/',expand=True)
df3 = pd.concat([df3,df3_3],axis=1)
df3.head()

,type,category,review,star,date,option,0,1,2
0,헤트라스,별점,디퓨저 좋아하는 사람이라 이것저것 많이 옴겨타다가 인스광고보고 혹시나하고 사봤어요....,평점3,22.07.19.,"옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향...","옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향...",None,None
1,헤트라스,별점,"일단..\n저한테는 좀 안맞는거같아요\n향이 너무 진하게 퍼지다보니 머리 아파서,\...",평점3,21.11.11.,"옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#01_디퓨저 앙쥬 50...","옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#01_디퓨저 앙쥬 50...",None,None
2,헤트라스,별점,스틱 2개꼽아도 향이 가득하다던데 첫날빼고는 그닥 효과를 못본거같아요 다른한개는 3...,평점3,22.02.14.,"옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#05_디퓨저 메모리아 ...","옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#05_디퓨저 메모리아 ...",None,None
3,헤트라스,별점,생각보다 지속력이 좋진 않은 듯\n초반 발향력은 매우 좋았음 그건 정말 ㅇㅈ\n퇴근...,평점3,21.09.07.,"옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 5...","옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 5...",None,None
4,헤트라스,별점,진짜 잘 닳아서 대용량으로 구매했어요. 발향이 항상 잘 되는 건 아니에요. 스틱 교...,평점3,22.08.02.,"옵션 선택=디퓨저 리필액 1L, 향기 선택=#08_리필액 북샵 1000ml, 1+1...","옵션 선택=디퓨저 리필액 1L, 향기 선택=#08_리필액 북샵 1000ml, 1+1...",None,None


In [21]:
# 전처리 후 크롤링한 3점 데이터의 갯수
len(df3)

376

In [22]:
# 웹페이지 열기

header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_13888/2638303677.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [23]:
# 2점 리뷰 가져오기
d.find_element(By.XPATH,category4).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
dates=[]
options=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    #sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            date=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[4]').text
            dates.append(date)
            option=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[5]').text
            options.append(option)
            #if j%2==0: #화면에 2개씩 보이도록 스크롤
            #    ELEMENT = d.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
            #    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review, star, date, option, "\n")
            cnt+=1 
        except: break
            
        #sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
        
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[12]').click()
            else : next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
#save()

페이지 1 

1 역시 향기나는 제품은 직접가서 맡아보고 사야함.
그리고 향기 나는 제품은 저가형 사는거 아님을 다시한번 느낌.

발향력이 좋다? 냄새가 잘퍼진다? 인정.
이건 진짜 인정.

근데 좋은 향기가 잘퍼져야 의미가 있는거 아닐까?

머리가 너무아파서 스틱 다뽑고 한개 꽂아놓고 그것도 뚜껑주변을 휴지로막음.

그래도 이거 있는 쪽은 냄새퍼짐.

이정도면 아무리 좋은향이어도 머리 아플듯.

근데 저가형 머리아픈향이라 발향력은 쓸데없이 좋아서 미쳐버릴뻔..

스틱1개꼽고 뚜껑밀봉하니까 그나마 좀 쓸만은 함. 평점2 21.12.28. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#08_디퓨저 아이솝 500ml 

2 첫구매시 발향이 좋아서 두번째 1+1 구매하였는데
하나는 정상이었는데 또 다른하나는 발향이 불량이었습니다. 냄새가 나질않아서
다시 세번째 1+1 구매시 플로랑에 얘기했더니
벨르 하나를 더 보내준다고해서
총 벨르향 2 우디 1 받았습니다.
막상 받아보니 또 청포도 발향이 약했습니다.
세번째 구매한것도 발향이 약했습니다.
저혼자 쓰는게아니라 회사에서 쓰는거라
회사사람들도 기존에 썼던건 냄새가 복도까지
퍼졌는데 최근에받은건 스틱에 코를 갖다대야
냄새가 난다고합니다.
저만 그런건가요?다른사람들은 문제가없는건지요?
사진은 세번째 구매시 받은건데
올립니다 평점2 22.06.16. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#03_디퓨저 벨르 500ml, 1+1 선택=#10_디퓨저 북샵 500ml 

3 다우니향이랑 북샵? 두개 샀어요 사실 향기뿜뿜이 너무 크게 적혀있어서 인테리어용으론 안예뻐서 ㅠ 고민 했는데 유튭님이 완전 강추해서 발향력때매 산건데 개인적으로 코코**보다 약해요 ㅠ 후기는 발향 진짜 잘된다 머리 아플지경이다 이러는데 저는 7평짜리 집인데도 발향이 안되는것 같더라구요 특히 북샵?서점향은 편백나무향이 좋긴한데 아리아(다우니)향보단 발향 엄청 약해요 향마다의 차이인건

36 첫 사용 일주일은 향이 잘나더니 그 이후론 약해져서 가까이 가야지 냄새가 나요
거실이 좀 넓은 편인데 하나만 놔둬서는 향이 아예 안나네요 광고에 속지마세요 평점2 22.03.05. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#01_디퓨저 앙쥬 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

37 누가 향이좋다고 ;;진짜 자연스러운향이 아니고 너무강해서 자다가 화장실에 놔둠 참고로 두개만꽂음

인위적인향 싸구려 향 이에요 싼맛에 사봣더니 평점2 22.11.02. 옵션 선택: 디퓨저 향기뿜뿜 1+1(55%할인) / 향기 선택: #03_디퓨저 벨르 500ml / 1+1 선택: #05_디퓨저 둘치스 500ml 

38 음... 솔직한 리뷰
향 좋아요 잘 퍼지는거 맞구요
근데 1/5 도 안줄었는데 어느순간부터 향이 거의 안나요;;; 그래서 있으나 마나 하네요 향이 한.. 열흘 갑니다 평점2 22.08.28. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#05_디퓨저 아이솝 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

39 한개로 30평대 커버 가능이라니 과장이 너무 심하네요
3개 꽂아서 화장실에 뒀는데도 향 잘 안나요
돈아까버서 걍 씁니다 평점2 22.02.23. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#09_디퓨저 가든파티 500ml 

40 첫발향은 너무 머리아프고 코찌르는 ㅜㅠ
독하게 퍼져요~~ ㅠㅠㅠ지금은 은은해 졌는데~~ 첨보단 향이 팍 죽었고
청포도향 추천많이 하던데 저는 너무 별로에요 ㅠ 고급진 향은 아니에요 평점2 22.02.22. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

페이지 3 

41 역시 향기나는 제품은 직접가서 맡아보고 사야함.
그리고 향기 나는 제품은 저가형 사는거 아님을 다시한번 느낌.

발향력이 좋다? 냄새가 잘퍼진다? 인정.
이건 진

96 광고 정도는 아닌거 같아요.. 평점2 22.11.11. 옵션 선택: 디퓨저 향기뿜뿜 1+1(55%할인) / 향기 선택: #01_디퓨저 앙쥬 500ml / 1+1 선택: #01_디퓨저 앙쥬 500ml 

97 향은좋은데..음 쫌그러네요 평점2 22.11.07. 옵션 선택: 디퓨저 향기뿜뿜 1+1(55%할인) / 향기 선택: #07_디퓨저 가든파티 500ml / 1+1 선택: #07_디퓨저 가든파티 500ml 

98 가격값 하는 제품이네요... 평점2 21.11.18. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#03_디퓨저 벨르 500ml 

99 향이.... 거의 없는듯해요.. 평점2 20.01.08. 옵션 선택=디퓨저 BEST3(65%할인), 향기 선택=BEST3세트(앙쥬+아리아+벨르), 1+1 선택=선택 안함 

100 향이 너무 독해서 눈아픔 평점2 21.10.11. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#02_디퓨저 아리아 500ml 

101 향이 생각보다 별로에요 평점2 21.12.29. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#09_디퓨저 가든파티 500ml, 1+1 선택=#09_디퓨저 가든파티 500ml 

102 향좋음 배송 느립니다... 평점2 21.09.16. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#08_디퓨저 아이솝 500ml, 1+1 선택=#08_디퓨저 아이솝 500ml 

페이지 8 

103 냄시가안나네요...안쓰고있어요.. 평점2 22.05.02. 옵션 선택=디퓨저 리필액 1L, 향기 선택=#01_리필액 앙쥬 1000ml, 1+1 선택=선택 안함 

페이지 9 

104 향기가 진하긴해요 ㅎㅎ 평점2 22.09.28. 옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

105 앙쥬는 그나마

In [24]:
# 2점 리뷰 데이터프레임으로 표현
df4=add_dataframe(name_,category_,reviews,stars,dates,options,cnt)
df4.head()

,type,category,review,star,date,option
1,헤트라스,별점,역시 향기나는 제품은 직접가서 맡아보고 사야함.\n그리고 향기 나는 제품은 저가형 ...,평점2,21.12.28.,"옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 5..."
2,헤트라스,별점,첫구매시 발향이 좋아서 두번째 1+1 구매하였는데\n하나는 정상이었는데 또 다른하나...,평점2,22.06.16.,"옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#03_디퓨저 벨르 50..."
3,헤트라스,별점,다우니향이랑 북샵? 두개 샀어요 사실 향기뿜뿜이 너무 크게 적혀있어서 인테리어용으론...,평점2,22.10.17.,"옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 5..."
4,헤트라스,별점,원래 별점 낮게주고 그러지 않는데 흠 솔직히 광고에는 이거 하나면 30평대 다 문제...,평점2,22.11.06.,"옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#07_디퓨저 가든파티 5..."
5,헤트라스,별점,용량도 크고 후기도 좋아서 구매했는데 도착해서 바로 사용해봤는데 스틱 5개 꽂아도 ...,평점2,22.03.22.,"옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 5..."


In [25]:
# 크롤링한 2점 데이터의 갯수
len(df4)

115

In [26]:
# 2점 데이터 전처리

df4 = df4.drop_duplicates()
df4 = df4.reset_index(drop=True)
df4_4 = df4['option'].str.split('/',expand=True)
df4 = pd.concat([df4,df4_4],axis=1)
df4.head()

,type,category,review,star,date,option,0,1,2
0,헤트라스,별점,역시 향기나는 제품은 직접가서 맡아보고 사야함.\n그리고 향기 나는 제품은 저가형 ...,평점2,21.12.28.,"옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 5...","옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_디퓨저 아리아 5...",None,None
1,헤트라스,별점,첫구매시 발향이 좋아서 두번째 1+1 구매하였는데\n하나는 정상이었는데 또 다른하나...,평점2,22.06.16.,"옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#03_디퓨저 벨르 50...","옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#03_디퓨저 벨르 50...",None,None
2,헤트라스,별점,다우니향이랑 북샵? 두개 샀어요 사실 향기뿜뿜이 너무 크게 적혀있어서 인테리어용으론...,평점2,22.10.17.,"옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 5...","옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 5...",None,None
3,헤트라스,별점,원래 별점 낮게주고 그러지 않는데 흠 솔직히 광고에는 이거 하나면 30평대 다 문제...,평점2,22.11.06.,"옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#07_디퓨저 가든파티 5...","옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#07_디퓨저 가든파티 5...",None,None
4,헤트라스,별점,용량도 크고 후기도 좋아서 구매했는데 도착해서 바로 사용해봤는데 스틱 5개 꽂아도 ...,평점2,22.03.22.,"옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 5...","옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 5...",None,None


In [27]:
# 전처리 후 크롤링한 2점 데이터의 갯수
len(df4)

94

In [28]:
# 웹페이지 열기

header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address,verify=False)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_13888/2638303677.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d = webdriver.Chrome('/Users/hamsongmin/Codestates/project_2/chromedriver') # webdriver = chrome
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.shopping.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [29]:
# 1점 리뷰 가져오기
d.find_element(By.XPATH,category5).click() #스크롤 건드리면 안됨
name_=name[0] 
category_=category[0]
reviews=[]
stars=[]
dates=[]
options=[]
cnt=1   #리뷰index
page=1
while True:
    j=1
    print ("페이지", page ,"\n") 
    #sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[2]/div[1]/p').text
            reviews.append(review)
            date=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[4]').text
            dates.append(date)
            option=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/ul/li['+str(j)+']/div[1]/span[5]').text
            options.append(option)
            #if j%2==0: #화면에 2개씩 보이도록 스크롤
            #    ELEMENT = d.find_element(By.XPATH,'/html/body/div[2]/div[2]/div[2]/div[3]/div[5]/div[2]/ul/li['+str(j)+']/div/div[2]')
            #    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, review, star, date, option, "\n")
            cnt+=1 
        except: break
            
        #sleep(2)
    
    if page<11:#page10까지 적용
        try: #리뷰의 마지막 페이지에서 error발생
            next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page)+']').click() 
            page +=1
        except: break #리뷰의 마지막 페이지에서 process 종료 
        
    else :
        try: #page11부터
            if page%10==0: next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a[12]').click()
            else : next_page=d.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[5]/div[3]/a['+str(page%10+2)+']').click()
            page+=1
        except: break 
            
#save()

페이지 1 

1 상품은 모르겠지만 CS가 개판입니다.
최초 주말 상품 주문 후 주소지 오류를 발견하였고, 일요일 주문취소를 하려고 보니 발송도 되지 않은 상품이 반품 밖에 되지 않아 주소지 오기재로 재 주문예정이라고 남기고 반품을 요청하였으나 월요일 저녁까지도 어떠한 답변이나 처리가 되지 않아 재차 문의 하니 화요일 상품이 배송되었다는 알람과 함께 발송되었으니 반품 하시라는 답변만 받았습니다. 상품 발송 전 문의와 철회를 요청했지만 모두 무시하고 발송후에 알아서 하란 식이였습니다. 이럴거면 문의는 왜 받는지 의문스럽네요. 덕분에 잘못된 주소지에 양해를 구하고 40여 킬로미터 거리를 운전하여 직접 다녀왔습니다. 상용해보고 좋으면 지속 구매의사가 있었으나 해당 업체는 더 이상 이용하지 않을 생각이며, 주변에도 권하지 않을 생각입니다. 평점5 22.03.22. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#10_디퓨저 북샵 500ml, 1+1 선택=#10_디퓨저 북샵 500ml 

2 이상하네요... 그전에 아리아랑 아이솝 주문해서 썼을땐 농담섞어서 향으로 암살수준이네 뭐네 했을정도로
향이 진해서 방에는 스틱 세개만 꼽아도 향이 진동을 했고
거실에도 광고처럼 발향력이 좋아서
주변에 추천하고 그랬는데
블랙체리향은 액만 흡수하고 발향력이 너무 안좋네요..
가까이 가서 코를 갖다대야지만
그래서 스틱도 7개나 꼽았는데도 향이안퍼지네요
향 종류의 문제인건지... 평점1 22.03.28. 옵션 선택=디퓨저 리필액 1L, 향기 선택=#02_리필액 아리아 1000ml, 1+1 선택=선택 안함 

3 배송요청사항에 원하는향 쓰면된다해서 쓴거같은데 전혀 안보시나봐요 원하는향은 단품으로지정한거빼고는 다 이상한.. 향
그리고 포장 내용물안흘러나오게는 못하나봐요 다 샜어요 제품을싸고있는 뽁뽁이를 만지는순간 손이 축축.. 한번써봤던 곳이라고 또 주문한거였는데 진짜 쫌.. 실망스럽네요 향도 뭔가 코를찌르는듯한.. 무튼 그닥입니다 평점1 20.05.29. 옵션 선택=디퓨저 BE

29 향이 좋고 진하고 잘퍼져요
다만 택배사 바꾸셔야 할듯 하네요
제품을 받은적이 없는데 배송완료에
상자는 누가 뜯었고 심지어 쓰던걸 다시 넣었는지
모리스는 양도 다르고 실리콘 뚜껑도 한번 뺐다넣은것처럼 헐거워서 톡 치니 빠지네요
이게 무슨 경우인지
용량많고 남이 쓰던걸 써야하는데 평점1 22.03.16. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#06_디퓨저 몰리스 500ml 

30 어디서 꽃향기가 난다는건지...
하수구 썩은냄새도 아니고...뭐 이상한 냄새가 온 집안을 뒤덮어서 몇번을 환기를 시켰는지...ㅠㅠ
심지어 애들도 집에서 하수구냄새난다고 뭐라고 하네요...싼게 비지떡이라고...제가 이상한 제품을 받은건지...역시 네이버리뷰보다는 지인추천으로 제품을 사야겠어요. 평점1 21.03.22. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#02_아리아 500ml+섬유스틱(5개), 1+1 선택=#05_메모리아 500ml+섬유스틱(5개) 

31 체리향 두번째 구매입니다
첫번째 구매시 화장실에 놓았는데 체리냄새가 진해서 거실까지 퍼질정도로 좋아서 재구매를 했는데 설치 첫날부터 냄새가 전혀나지않아 스틱을 다섯개 꽂았는데도 나질않네요. 병에다 코를 갖다되어도 체리향은 전혀없네요
향이 없을수도 있나요? 평점1 22.01.02. 옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#01_디퓨저 앙쥬 500ml, 1+1 선택=선택 안함 

32 처음 주문 했을때 거실에 스틱 5개로도 발향력이 엄청났는데..
두세번 주문해 쓰는데 점점 덜 하네요..
스틱갯수를 늘리나는데..스틱 추가해서 꽂아도 어림없음..거실에도 안되 작은방에도 스틱 5개로는 향이 안나는건 무엇때문인지... 평점1 21.11.28. 옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#03_디퓨저 벨르 500ml, 1+1 선택=선택 안함 

33 머리가 너무 아파요.....ㅠㅠ
청포도? 포도향... 후기보고 샀는데


81 광고에 속았네요 냄새 1도 안퍼집니다. 평점1 21.08.12. 옵션 선택=디퓨저 향기뿜뿜 1+1(58%할인), 향기 선택=#01_디퓨저 앙쥬 500ml, 1+1 선택=#05_디퓨저 메모리아 500ml 

82 향이 너무 싸구려같아요 단향은 너무심하구요 향이좋다는 평보고샀는데 다...알바인가요? 무더위에 달달한향은...날파리들이 득실거릴것만같은 향이네요;; 고급진향이아예아니어서 기대이하입니다 평점1 21.08.10. 옵션 선택=디퓨저 3종세트(60%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

페이지 7 

83 발향 진하다고 해서 구입했는데
하나도..안나요 평점1 22.10.12. 옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

84 향이 완젼싸구려 뜯자마자 머리아파서 다버림
아직도 냄새가나서 머리 너무아품 평점1 22.11.27. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#06_디퓨저 아이솝 500ml 

85 발향력 거지같음 풍자믿고샀는데 참.. 평점1 21.12.29. 옵션 선택=디퓨저 리필액 1L, 향기 선택=#02_리필액 아리아 1000ml, 1+1 선택=선택 안함 

86 발향이 좋다기에 써봤는데
전혀… 평점1 22.03.24. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#01_디퓨저 앙쥬 500ml, 1+1 선택=#01_디퓨저 앙쥬 500ml 

87 두개 구매했는데 왜 양이 틀릴까요... 평점1 22.09.20. 옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#02_디퓨저 아리아 500ml, 1+1 선택=#05_디퓨저 둘치스 500ml 

88 여러향을구매했는데 향이별로였어요 평점1 21.09.11. 옵션 선택=디퓨저 3종세트(60%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향 작성), 1+1 선택=선택 안함 

89 향이하나도 안

In [30]:
# 1점 리뷰 데이터프레임으로 표현
df5=add_dataframe(name_,category_,reviews,stars,dates,options,cnt)
df5.head()

,type,category,review,star,date,option
1,헤트라스,별점,상품은 모르겠지만 CS가 개판입니다.\n최초 주말 상품 주문 후 주소지 오류를 발견...,평점5,22.03.22.,"옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#10_디퓨저 북샵 50..."
2,헤트라스,별점,이상하네요... 그전에 아리아랑 아이솝 주문해서 썼을땐 농담섞어서 향으로 암살수준이...,평점1,22.03.28.,"옵션 선택=디퓨저 리필액 1L, 향기 선택=#02_리필액 아리아 1000ml, 1+..."
3,헤트라스,별점,배송요청사항에 원하는향 쓰면된다해서 쓴거같은데 전혀 안보시나봐요 원하는향은 단품으로...,평점1,20.05.29.,"옵션 선택=디퓨저 BEST3(65%할인), 향기 선택=BEST3세트(앙쥬+아리아+벨..."
4,헤트라스,별점,제품받아서 사용해봤는데 다른분들은 발향력이 좋다고하는데\n저는 제가 사용하던 다른 ...,평점1,22.03.10.,"옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#01_디퓨저 앙쥬 500..."
5,헤트라스,별점,아리아2+1이랑 리필1리터 구매했습니다\n광고에는 발향력이 좋다고 해서 구매를 했습...,평점1,22.07.18.,"옵션 선택=디퓨저 리필액 1L, 향기 선택=#02_리필액 아리아 1000ml, 1+..."


In [31]:
# 전처리 후 크롤링한 1점 데이터의 갯수
len(df5)

124

In [32]:
# 1점 데이터 전처리

df5 = df5.drop_duplicates()
df5 = df5.reset_index(drop=True)
df5_5 = df5['option'].str.split('/',expand=True)
df5 = pd.concat([df5,df5_5],axis=1)
df5.head()

,type,category,review,star,date,option,0,1,2
0,헤트라스,별점,상품은 모르겠지만 CS가 개판입니다.\n최초 주말 상품 주문 후 주소지 오류를 발견...,평점5,22.03.22.,"옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#10_디퓨저 북샵 50...","옵션 선택=디퓨저 향기뿜뿜 1+1(55%할인), 향기 선택=#10_디퓨저 북샵 50...",None,None
1,헤트라스,별점,이상하네요... 그전에 아리아랑 아이솝 주문해서 썼을땐 농담섞어서 향으로 암살수준이...,평점1,22.03.28.,"옵션 선택=디퓨저 리필액 1L, 향기 선택=#02_리필액 아리아 1000ml, 1+...","옵션 선택=디퓨저 리필액 1L, 향기 선택=#02_리필액 아리아 1000ml, 1+...",None,None
2,헤트라스,별점,배송요청사항에 원하는향 쓰면된다해서 쓴거같은데 전혀 안보시나봐요 원하는향은 단품으로...,평점1,20.05.29.,"옵션 선택=디퓨저 BEST3(65%할인), 향기 선택=BEST3세트(앙쥬+아리아+벨...","옵션 선택=디퓨저 BEST3(65%할인), 향기 선택=BEST3세트(앙쥬+아리아+벨...",None,None
3,헤트라스,별점,제품받아서 사용해봤는데 다른분들은 발향력이 좋다고하는데\n저는 제가 사용하던 다른 ...,평점1,22.03.10.,"옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#01_디퓨저 앙쥬 500...","옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#01_디퓨저 앙쥬 500...",None,None
4,헤트라스,별점,아리아2+1이랑 리필1리터 구매했습니다\n광고에는 발향력이 좋다고 해서 구매를 했습...,평점1,22.07.18.,"옵션 선택=디퓨저 리필액 1L, 향기 선택=#02_리필액 아리아 1000ml, 1+...","옵션 선택=디퓨저 리필액 1L, 향기 선택=#02_리필액 아리아 1000ml, 1+...",None,None


In [33]:
# 전처리 후 크롤링한 1점 데이터의 갯수
len(df5)

124

In [34]:
# 5점~1점 데이터합치기
df_ascent = pd.concat([df1,df2,df3,df4,df5])
df_ascent.head()

,type,category,review,star,date,option,0,1,2,3
0,헤트라스,별점,"제가 캔들,디퓨저 만드는 사람입니다~\n디퓨져는 사서는 써본적없고 받아서만 써보았어...",평점5,21.06.24.,"옵션 선택=디퓨저 리필액 1L, 향기 선택=#02_리필액 아리아 1L, 1+1 선택...","옵션 선택=디퓨저 리필액 1L, 향기 선택=#02_리필액 아리아 1L, 1+1 선택...",None,None,None
1,헤트라스,별점,디퓨저 좋아하는 사람이라 이것저것 많이 옴겨타다가 인스광고보고 혹시나하고 사봤어요....,평점3,22.07.19.,"옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향...","옵션 선택=디퓨저 6종세트(60%할인), 향기 선택=디퓨저 6종세트(배송메세지에 향...",None,None,None
2,헤트라스,별점,가격도 저렴하고\n작은 통에 소분해서 여러군데 놔뒀는데\n일단 좁은 곳에서는 발향이...,평점5,22.08.12.,"옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#02_디퓨저 아리아 50...","옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#02_디퓨저 아리아 50...",None,None,None
3,헤트라스,별점,처음 구매하는거라 향이랑 발향력 볼려구 단품주문햇는데.. 박스에서 이미 디퓨저향이 ...,평점5,22.10.19.,"옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#02_디퓨저 아리아 50...","옵션 선택=디퓨저 향기뿜뿜 단품(54%할인), 향기 선택=#02_디퓨저 아리아 50...",None,None,None
4,헤트라스,별점,풍자님이 알려주셔서 2주전에 구매했는데\n\n진짜 거짓말이 아니고 다른 유명하다는 ...,평점5,21.11.29.,"옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향...","옵션 선택=디퓨저 3종세트(58%할인), 향기 선택=디퓨저 3종세트(배송메세지에 향...",None,None,None


In [35]:
# 1점~5점 데이터의 총 갯수
len(df_ascent)

3356

In [36]:
### CSV파일로 저장
df_ascent.to_csv('플로랑향기뿜뿜_평점별리뷰.csv')